# Exercise 03

## Aim of the exercise:

We will learn to:

- use OpenMM to simulate a protein in water
- understand the steps for setting up a simulation

This exercise is based on the official tutorials of [OpenMM](https://github.com/openmm/openmm_workshop_july2023) and [MDAnalysis](https://userguide.mdanalysis.org/stable/examples/quickstart.html).

<img src="https://github.com/yerkoescalona/structural_bioinformatics/blob/main/ex03/openmm_logo.png?raw=1" width="50"/>

# OpenMM

OpenMM is a versatile and high-performance toolkit designed for molecular simulation. It stands out due to its unique blend of features, which include extreme flexibility, openness, and exceptional performance, particularly on modern GPUs.

Summary of Contents:
1. **Setup Environment**: Instructions on preparing the necessary computational environment.
2. **Download Protein File**: Steps to obtain the required protein structure file.
3. **Load a PDB File into OpenMM**: Tutorial on how to import and use a PDB file within OpenMM.
4. **Choose the Force-Field**: Guidance on selecting appropriate force-fields for the simulation.
5. **Solvate the Protein with Water and Ions**: Process of adding water and ions to the protein environment.
6. **Setup System and Integrator**: Instructions on initializing the simulation system and integrator.
7. **Run Local Minimization**: Steps for performing local energy minimization of the system.
8. **Setup Reporting**: How to configure reporting tools for monitoring simulation progress.
9. **Run NVT Equilibration**: Conducting NVT (constant Number, Volume, Temperature) equilibration simulations.
10. **Run NPT Production Molecular Dynamics**: Executing NPT (constant Number, Pressure, Temperature) production simulations.
11. **Basic Analysis**: Introduction to fundamental analysis techniques for simulation data.
12. **How to Use Checkpoints**: Instructions on setting up and utilizing checkpoints for long-term simulations.
13. **Visualization**: Tips and tools for visualizing the simulation results.



## Setup
**First try and change runtime type to GPU!**  
Click "runtime">"change runtime type" and select "GPU" from the "Hardware accelerator" dropdown menu.
CPU works, but it is slower.


In [1]:
if "google.colab" in str(get_ipython()):
    print("Running on colab")
    print("Installing required packages...")
    !pip install -q openmm==8.2.0 mdanalysis==2.10.0 py3dmol==2.4.0
    print("Installation complete!")
else:
    print("Not running on colab.")
    print("Make sure you have all required packages installed in your environment!")


Running on colab
Installing required packages...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.9 MB/s eta 0:00:00
Installation complete!


**Note:** During this step on Colab the kernel will be restarted. This will produce the error message:
"Your session crashed for an unknown reason. " This is normal and you can safely ignore it.

**Note:** Installing the packages will take several minutes!

In [2]:
# check the installation
!python -m openmm.testInstallation


OpenMM Version: 8.2
Git Revision: 53770948682c40bd460b39830d4e0f0fd3a4b868

There are 3 Platforms available:

1 Reference - Successfully computed forces
2 CPU - Successfully computed forces
1 warning generated.
1 warning generated.
3 OpenCL - Successfully computed forces

Median difference in forces between platforms:

Reference vs. CPU: 6.30005e-06
Reference vs. OpenCL: 6.75018e-06
CPU vs. OpenCL: 7.67879e-07

All differences are within tolerance.


## Protein to Study: Villin headpiece


Villin is a popular protein for molecular dynamics studies, and several papers on its folding have been published. Additionally, it has been found that the proximity of three phenylalanines that comprise the hydrophobic core is important for the correct structure.

This is small fast folding protein commonly used as a toy system. Note that this PDB file has been cleaned up and is ready for use in OpenMM. If you try and use a PDB file directly from the protein data bank you may encounter errors. Please look at the [OpenMM FAQs](https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions) and [PDBfixer](https://github.com/openmm/pdbfixer).

![villin](https://github.com/yerkoescalona/structural_bioinformatics/blob/main/ex03/villin.png?raw=1)
**Figure**. Villin headpiece protein.


In [3]:
if "google.colab" in str(get_ipython()):
    print("Running on colab")
    !wget https://raw.githubusercontent.com/yerkoescalona/structural_bioinformatics/main/ex03/villin.pdb
else:
    print("Not running on colab.")
    print("You should have villin.pdb in your path!")

Running on colab
--2025-11-23 08:28:19--  https://raw.githubusercontent.com/yerkoescalona/structural_bioinformatics/main/ex03/villin.pdb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39178 (38K) [text/plain]
Saving to: ‘villin.pdb’

villin.pdb          100%[===================>]  38.26K  --.-KB/s    in 0.003s  

2025-11-23 08:28:19 (13.1 MB/s) - ‘villin.pdb’ saved [39178/39178]



## Load the PDB file into OpenMM
<a id="load"></a>

First we need to import OpenMM.
We then then load in the PDB file using the [PDBFile](http://docs.openmm.org/latest/api-python/generated/openmm.app.pdbfile.PDBFile.html#openmm.app.pdbfile.PDBFile) class.

In [4]:
from openmm.app import (
    ForceField,
    Modeller,
    HBonds,
    Simulation,
    DCDReporter,
    StateDataReporter,
    PME,
    PDBFile,
    CheckpointReporter,
)
from openmm import LangevinMiddleIntegrator, MonteCarloBarostat, XmlSerializer
from openmm.unit import kelvin, nanometer, picosecond, picoseconds, bar, seconds
from sys import stdout

# load in the pdb file
pdb = PDBFile("villin.pdb")

`PDBFile('file_name.pdb')` loads the PDB file from disk and puts the information into a `PDBFile` object which we have assign to the variable `pdb`. The object contains the molecular topology (atom names, residue types, bonds etc) and the atomic positions. These can be accessed as `pdb.topology` and `pdb.positions`. Take a look at the [API documentation](http://docs.openmm.org/latest/api-python/generated/openmm.app.pdbfile.PDBFile.html#openmm.app.pdbfile.PDBFile). All OpenMM classes have documentation available on the Python API reference: http://docs.openmm.org/latest/api-python/.

## Define the force field
<a id="ff"></a>

We need to define the forcefield we want to use. We will use the Amber14 forcefield and the TIP3P-FB water model. You can find out about all the forcefields available by default in OpenMM in the [documentation](http://docs.openmm.org/latest/userguide/application/02_running_sims.html?highlight=forcefield#force-fields).

In [5]:
# Specify the forcefield
forcefield = ForceField("amber14-all.xml", "amber14/tip3pfb.xml")

Force fields are defined by XML files. The line above loads in specified files. You can look at them in the OpenMM source code, e.g. [`amber14/tip3pfb.xml`](https://github.com/openmm/openmm/blob/master/wrappers/python/openmm/app/data/amber14/tip3pfb.xml). It is possible to create your own XML force field file. You can find details in the [user guide](http://docs.openmm.org/latest/userguide/application/05_creating_ffs.html#creating-force-fields).

## Solvate
<a id="solvate"></a>

We can use the [`Modeller`](http://docs.openmm.org/latest/userguide/application/03_model_building_editing.html#model-building-and-editing) class to solvate the protein in a waterbox.

In [6]:
# create a Modeller object
modeller = Modeller(pdb.topology, pdb.positions)


# Solvate the protein in a box of water
modeller.addSolvent(forcefield, padding=1.0 * nanometer)

This command creates a box that has edges at least 1nm away from the solute and fills it with water molecules. Additionally, it adds in the required number of CL- and Na+ ions to make the system charge neutral. Optionally, you can specify the ion concentration as an argument to [`addSolvent`](http://docs.openmm.org/latest/api-python/generated/openmm.app.modeller.Modeller.html#openmm.app.modeller.Modeller.addSolvent).

Note that the `nanometer` variable is a unit definition that was imported from `openmm.unit`. This is an example of the powerful units tracking and automatic conversion facility built into the OpenMM Python API that makes specifying unit-bearing quantities convenient and less error-prone. We could have equivalently specified `10*angstrom` instead of `1*nanometer` and achieved the same result. You can read more about the units library [here](http://docs.openmm.org/latest/userguide/library/05_languages_not_cpp.html#units-and-dimensional-analysis).


## Setup system and Integrator
<a id='system'></a>

We now need to combine our molecular topology and the forcefield to create a complete description of the system. This is done using the [`ForceField`](http://docs.openmm.org/latest/api-python/generated/openmm.app.forcefield.ForceField.html#forcefield) object’s [`createSystem()`](http://docs.openmm.org/latest/api-python/generated/openmm.app.forcefield.ForceField.html#openmm.app.forcefield.ForceField.createSystem) method. We then create the integrator, and combine the integrator and system to create the Simulation object. Finally we set the initial atomic positions.

In [7]:
# Create a system. Here we define some forcefield settings such as the nonbonded method
system = forcefield.createSystem(
    modeller.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=1.0 * nanometer,
    constraints=HBonds,
)

# Define the integrator. The Langevin integrator is also a thermostat
integrator = LangevinMiddleIntegrator(300 * kelvin, 1 / picosecond, 0.004 * picoseconds)

# Create the Simulation
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

The [System](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.System.html#system) is an object than contains the complete mathematical description of the system we want to simulate. It contains four key bits of information:
 - The set of particles in the simulation
 - The forces acting on them
 - Details of any constraints
 - The dimensions of the periodic box

The `integrator` advances the equations of motion. There are a variety of [integrators available in OpenMM](http://docs.openmm.org/latest/api-python/library.html#integrators). We are using the [LangevinMiddleIntegrator](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.LangevinMiddleIntegrator.html#openmm.openmm.LangevinMiddleIntegrator) which performs Langevin dynamics.

The [`Simulation`](http://docs.openmm.org/latest/api-python/generated/openmm.app.simulation.Simulation.html#openmm.app.simulation.Simulation) object manages all the process involved in running a simulation, such as advancing time and writing output.


## Local energy minimization
<a id="minim"></a>

It is a good idea to run local energy minimization at the start of a simulation because the coordinates in starting configuration file might produce very large forces.

Due to how the minimizer is implemented it will not print out information during the run. You will need to be patient and wait for it to complete. This minimization step should take ~1 minute on CPU and a few seconds on GPU.

In [8]:
print("Minimizing energy")
simulation.minimizeEnergy()

Minimizing energy


## Setup reporting
<a id="reporting"></a>

To get output from a simulation you need to add "reporters". We use [`DCDReporter`](http://docs.openmm.org/latest/api-python/generated/openmm.app.dcdreporter.DCDReporter.html) to write the coordinates every 1000 timesteps to 'traj.dcd' and we use [`StateDataReporter`](http://docs.openmm.org/development/api-python/generated/openmm.app.statedatareporter.StateDataReporter.html) to print the timestep, potential energy, temperature, and volume to the screen; and the same to a file called 'md_log.txt'. The Simulation object contains a list of reporters in `simulation.reporters` and we use the append method to add the reporters to it.

In [9]:
# Write trajectory to a file called traj.dcd every 1000 steps
simulation.reporters.append(DCDReporter("traj.dcd", 1000))

# Print state information to the screen every 1000 steps
simulation.reporters.append(
    StateDataReporter(
        stdout, 1000, step=True, potentialEnergy=True, temperature=True, volume=True
    )
)

# Print the same info to a log file every 100 steps
simulation.reporters.append(
    StateDataReporter(
        "md_log.txt",
        100,
        step=True,
        potentialEnergy=True,
        temperature=True,
        volume=True,
    )
)


## NVT equilibration
<a id=nvt></a>

We are using a Langevin integrator which means we are simulating in the NVT ensemble. To equilibrate the temperature we just need to run the simulation for a number of timesteps.

In [10]:
print("Running NVT")
simulation.step(1000)

Running NVT
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
1000,-109188.56963314128,289.50404556910365,75.907156543232


## NPT production MD
<a id=npt></a>

To run our simulation in the NPT ensemble we need to add in a barostat to control the pressure. We can use [`MonteCarloBarostat`](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.MonteCarloBarostat.html#openmm.openmm.MonteCarloBarostat). The parameters are the pressure (1 bar) and the temperature (300 K). The barostat assumes the simulation is being run at constant temperature, but it does not itself do anything to regulate the temperature. It is therefore critical that you always use it along with a Langevin integrator or Andersen thermostat, and that you specify the same temperature for both the barostat and the integrator or thermostat. Otherwise, you will get incorrect results.

In [11]:
system.addForce(MonteCarloBarostat(1 * bar, 300 * kelvin))

# It is important to call the reinitialize method on the simulation
# otherwise the modifications will not be applied.
simulation.context.reinitialize(preserveState=True)

We then run the simulation for 10000 steps.

In [12]:
print("Running NPT")
simulation.step(10000)

Running NPT
2000,-109160.80338642758,297.2658227324499,70.90898700659703
3000,-109513.87892107945,302.07136397889275,69.15036222345901
4000,-109323.75624529371,304.0009167011169,68.99575482831848
5000,-109966.10969872714,302.7604686519877,68.67797658022225
6000,-110026.03470147215,304.48523336437023,68.81673919138476
7000,-110102.82172992796,306.5380619782874,69.14756019768873
8000,-110026.65961854579,304.8893173397167,68.87666225574266
9000,-109606.40415810613,303.2252658495336,69.2557113862849
10000,-109799.29888437025,295.60190208230136,68.73353459142562
11000,-109653.58983016008,302.6362615001649,68.3257916150886


## Analysis
<a id=analysis></a>

We can now do some basic analysis using Python. We will plot the time evolution of the potential energy, temperature, and box volume. Remember that OpenMM itself is primarily an MD engine, for in-depth analysis of your simulations you can use other python packages such as [MDtraj](https://www.mdtraj.org/), or [MDAnalysis](https://www.mdanalysis.org/).


In [13]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set plotly as the plotting backend for pandas
pd.options.plotting.backend = "plotly"

md_log_df = pd.read_csv("md_log.txt", delimiter=",", header=0)
md_log_df.rename(columns={'#"Step"': "Step"}, inplace=True)

# Create subplots with plotly
fig = make_subplots(
    rows=3,
    cols=1,
    subplot_titles=("Potential Energy", "Temperature", "Box Volume"),
    vertical_spacing=0.1,
)

fig.add_trace(
    go.Scatter(
        x=md_log_df["Step"],
        y=md_log_df["Potential Energy (kJ/mole)"],
        name="Potential Energy",
        mode="lines",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=md_log_df["Step"],
        y=md_log_df["Temperature (K)"],
        name="Temperature",
        mode="lines",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=md_log_df["Step"],
        y=md_log_df["Box Volume (nm^3)"],
        name="Box Volume",
        mode="lines",
    ),
    row=3,
    col=1,
)

fig.update_xaxes(title_text="Step", row=3, col=1)
fig.update_yaxes(title_text="Potential Energy (kJ/mole)", row=1, col=1)
fig.update_yaxes(title_text="Temperature (K)", row=2, col=1)
fig.update_yaxes(title_text="Box Volume (nm^3)", row=3, col=1)

fig.update_layout(height=800, showlegend=False)
fig.show()

## Checkpointing
<a id=checkpoints></a>

When you run long simulations it is useful to be able to save checkpoints. This means you can restart them in the case of a crash. Or it means you can resume them if you need to fit within the time constraints of a HPC job scheduler.

To run a resume a simulation we need to have three files saved to disk that we can load in:
1. The topology - this will be a PDB file of our solvated system.
2. A serialized `System` -  this is an xml file that contains the forcefield settings.
3. A checkpoint file - this is a binary file that contains the positions, velocities, box vectors, and other internal data such as the states of random number generators.

The first two only need to be saved once because they are constant throughout the simulation. The checkpoint file needs to be saved frequently. You can then resume a simulation from the timestep when the checkpoint file was last saved.


### Setup the checkpoint

We will create the topology file using `PDBFile` to write a PDB file of the system. We will use `XmlSerializer` of save the serialized system to an xml file. And we will use `CheckpointReporter` to regularly create checkpoint files.

In [14]:
# Save the toplogy as a PDB file.
with open("topology.pdb", "w") as output:
    PDBFile.writeFile(
        simulation.topology,
        simulation.context.getState(getPositions=True).getPositions(),
        output,
    )

# save a serialized version of the system. This stores the forcefield parameters.
with open("system.xml", "w") as output:
    output.write(XmlSerializer.serialize(system))

# Setup a checkpoint reporter. This stores the positions, velocities, and box vectors. It will save
# a checkpoint every 1000 timesteps.
simulation.reporters.append(CheckpointReporter("checkpoint.chk", 1000))


In [15]:
import py3Dmol

# Visualize the initial snapshot (topology.pdb) - Full system
print("Initial Structure (Full System - Protein + Water + Ions):")
view = py3Dmol.view(width=800, height=500)
with open("topology.pdb") as f:
    pdb_data = f.read()
    view.addModel(pdb_data, "pdb")

# Style: Protein as cartoon, water/ions as spheres
view.setStyle({"model": 0}, {"cartoon": {"color": "spectrum"}})
view.addStyle({"resn": "HOH"}, {"sphere": {"radius": 0.4}})
view.addStyle({"resn": ["NA", "CL"]}, {"sphere": {"radius": 1, "color": "yellow"}})
view.zoomTo()
view.show()

Initial Structure (Full System - Protein + Water + Ions):


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Visualize the initial structure

We can use `py3Dmol` to visualize the initial structure (topology.pdb) directly in the notebook.

`CheckpointReporter` saves periodic checkpoints of a simulation. The checkpoints will overwrite one another - only the last checkpoint will be saved in the file. Loading a checkpoint will restore a simulation to a reasonably close, but usually not identical, state to when it was written. The checkpoint contains data that is highly specific to the System, Platform, and the hardware and software of the computer it was created on. If you try and load it on a computer with different hardware it is likely to fail. Checkpoints created with different versions of OpenMM are often incompatible.

For a more portable way of saving the state of a simulation you can save the checkpoint as an xml state file. Read the [API docs](http://docs.openmm.org/development/api-python/generated/openmm.app.checkpointreporter.CheckpointReporter.html) for more information.

### Running for a set time limit

We can run for a set amount of wall-clock time using the [`runForClockTime`](http://docs.openmm.org/latest/api-python/generated/openmm.app.simulation.Simulation.html#openmm.app.simulation.Simulation.runForClockTime) method. By [wall-clock](https://en.wikipedia.org/wiki/Elapsed_real_time) time we mean the actual time a program runs for as measured by looking at a clock on a wall (or a watch, or a timer etc) as opposed to the simulated time.


In [16]:
# run for 30 seconds
simulation.runForClockTime(30.0 * seconds)

12000,-109852.60788433562,301.9452953375096,69.457269295956
13000,-109494.45948709501,302.72333438396663,69.48780090211399
14000,-110118.29909279518,297.41821930945156,68.96498746033663
15000,-110016.86540064786,293.7980664411768,68.79739174105214
16000,-110341.47161438217,299.3378744135861,69.6595500239948
17000,-109934.86567684001,296.9586434812293,69.0844048045521
18000,-109567.41353940888,304.78456510912247,68.95821816798556
19000,-109879.20699438697,297.7992246216022,69.30261629113606
20000,-109390.29115006526,301.70132453753996,69.07142598239525
21000,-110169.23026078596,301.7986255309334,69.48495102955128
22000,-110279.83669608191,299.7342127556803,68.97918027922195
23000,-109707.15291620651,300.0061952201768,69.00916321826567
24000,-109370.68828674493,298.6013692454634,68.93307486715003
25000,-109459.47850268643,297.0412407647231,68.84106578932833
26000,-110682.08348724747,298.5331791922856,68.74419054358931
27000,-110002.4594420187,297.3277013353718,70.15191485619438
28000,-10

### Resume from a checkpoint

We now have the required files 'topology.pdb', 'system.xml', and 'checkpoint.chk'. We will need to load them in so we can resume the simulation from the last checkpoint. Note that we have to define the integrator again as well as the simulation reporters. Furthermore, we have set the `append=True` flag to the DCD and StateData reporters.

You will need to add a line of code to make the simulation run for 30 seconds of wall time.

In [39]:
pdb = PDBFile("topology.pdb")

with open("system.xml") as input:
    system = XmlSerializer.deserialize(input.read())

# Define the integrator.
integrator = LangevinMiddleIntegrator(300 * kelvin, 1 / picosecond, 0.004 * picoseconds)

# Create the Simulation
simulation = Simulation(pdb.topology, system, integrator)

# set the positions, velocities, and box vectors from the checkpoint file
simulation.loadCheckpoint("checkpoint.chk")

# We still need to define the reporters again

# Write trajectory to a file called traj.dcd every 1000 steps
simulation.reporters.append(DCDReporter("traj.dcd", 1000, append=True))

# Print state information to the screen every 1000 steps
simulation.reporters.append(
    StateDataReporter(
        stdout, 1000, step=True, potentialEnergy=True, temperature=True, volume=True
    )
)

# Print the same info to a log file every 100 steps
simulation.reporters.append(
    StateDataReporter(
        "md_log.txt",
        100,
        step=True,
        potentialEnergy=True,
        temperature=True,
        volume=True,
        append=True,
    )
)


# Setup a checkpoint reporter. This stores the positions, velocities, and box vectors.
simulation.reporters.append(CheckpointReporter("checkpoint.chk", 1000))


# write the code to run for 30 seconds of wall clock time
simulation.runForClockTime(30.0 * seconds)


#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
306000,-110398.83802084468,306.73291756859663,69.45773715489995
307000,-109526.53006580856,303.64482850630213,68.9062054867517
308000,-110174.18977946573,301.20328766193325,69.2358393140299
309000,-109906.3375767679,305.1664227891954,69.2047082265759
310000,-109980.1155231134,306.71948842237487,68.88623327771414
311000,-110406.0597824686,304.30518602756007,69.2806186088209
312000,-109786.44450631249,306.41198045558605,69.1333604955813
313000,-109826.8834225951,305.3522958630295,68.90042606000236
314000,-109284.45701989823,298.76461734007887,69.77074606622564
315000,-110057.51483680797,299.1031351953568,69.15780878965255
316000,-109849.84396067256,303.8163709386528,68.87507141564774
317000,-110054.58086592407,292.48713355493874,69.51209280518428
318000,-109841.4392456988,302.22150364503267,68.8973768432472
319000,-109882.07734208577,310.8314672785827,69.37300483711816
320000,-109591.42236633756,298.71682578171493

### Resume multiple times

We can practice resuming multiple times. This is something you might have to do to fit a long simulation within the limits of a HPC job scheduler.

You will need to add the code to create the `Simulation` object.

In [41]:
for i in range(3):
    print("Resuming from checkpoint iteration = ", i)

    pdb = PDBFile("topology.pdb")

    with open("system.xml") as input:
        system = XmlSerializer.deserialize(input.read())

    # Define the integrator.
    integrator = LangevinMiddleIntegrator(
        300 * kelvin, 1 / picosecond, 0.004 * picoseconds
    )

    # Create the Simulation
    # write the code to create the simulation object
    simulation = Simulation(pdb.topology, system, integrator)

    # set the positions, velocities, and box vectors from the checkpoint file
    simulation.loadCheckpoint("checkpoint.chk")

    # We still need to define the reporters again

    # Write trajectory to a file called traj.dcd every 1000 steps
    simulation.reporters.append(DCDReporter("traj.dcd", 1000, append=True))

    # Print state information to the screen every 1000 steps
    simulation.reporters.append(
        StateDataReporter(
            stdout, 1000, step=True, potentialEnergy=True, temperature=True, volume=True
        )
    )

    # Print the same info to a log file every 100 steps
    simulation.reporters.append(
        StateDataReporter(
            "md_log.txt",
            100,
            step=True,
            potentialEnergy=True,
            temperature=True,
            volume=True,
            append=True,
        )
    )

    # Setup a checkpoint reporter. This stores the positions, velocities, and box vectors.
    simulation.reporters.append(CheckpointReporter("checkpoint.chk", 1000))

    # run for 30 seconds
    simulation.runForClockTime(30.0 * seconds)


Resuming from checkpoint iteration =  0
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
366000,-109436.7898124734,297.2825329390875,69.05991209372873
367000,-109557.87176567398,298.6254602999673,69.46961653678453
368000,-109750.71265288192,301.79097358750096,68.49899446655441
369000,-109164.75113453018,302.2056104996623,68.42469376227199
370000,-110052.44959289621,299.7282556985691,68.55936670252176
371000,-109245.22789527103,295.58890005365515,69.27488765567254
372000,-109248.52303820226,297.43939012391587,68.86372492176311
373000,-110462.537003176,297.9771468637934,68.94394995936804
374000,-110102.9277991454,305.5545739349874,69.17209913785763
375000,-109908.81876799389,299.54318245832263,68.98776480845169
376000,-110075.4026535651,303.8325309324376,68.78443289077201
377000,-110070.04476565018,299.7577080567332,69.0932032631686
378000,-109904.16782173613,294.3322877255192,69.20285931167133
379000,-109770.28826829756,305.92041075758084,68.99422289469348
3800

### Analysis

we can redo the analysis on the longer trajectory.

In [19]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set plotly as the plotting backend for pandas
pd.options.plotting.backend = "plotly"

md_log_df = pd.read_csv("md_log.txt", delimiter=",", header=0)
md_log_df.rename(columns={'#"Step"': "Step"}, inplace=True)

# Create subplots with plotly
fig = make_subplots(
    rows=3,
    cols=1,
    subplot_titles=("Potential Energy", "Temperature", "Box Volume"),
    vertical_spacing=0.1,
)

fig.add_trace(
    go.Scatter(
        x=md_log_df["Step"],
        y=md_log_df["Potential Energy (kJ/mole)"],
        name="Potential Energy",
        mode="lines",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=md_log_df["Step"],
        y=md_log_df["Temperature (K)"],
        name="Temperature",
        mode="lines",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=md_log_df["Step"],
        y=md_log_df["Box Volume (nm^3)"],
        name="Box Volume",
        mode="lines",
    ),
    row=3,
    col=1,
)

fig.update_xaxes(title_text="Step", row=3, col=1)
fig.update_yaxes(title_text="Potential Energy (kJ/mole)", row=1, col=1)
fig.update_yaxes(title_text="Temperature (K)", row=2, col=1)
fig.update_yaxes(title_text="Box Volume (nm^3)", row=3, col=1)

fig.update_layout(height=800, showlegend=False)
fig.show()

# MDAnalysis

[MDAnalysis](https://www.mdanalysis.org/) is a Python library for analyzing molecular dynamics (MD) simulations. It supports various MD formats for reading and writing trajectories and atom selections. Key features include reading particle-based trajectories, accessing atomic coordinates via NumPy arrays, powerful atom selection commands, and the ability to manipulate and write out trajectories. This makes it a flexible and efficient tool for complex MD analysis tasks.


In [20]:
import MDAnalysis as mda

MDAnalysis is a Python package for accessing and analyzing molecular dynamics trajectory data. Its core data structures include:

- **Atom**: Represents a particle, even if it's a coarse-grained bead.
- **AtomGroup**: A crucial class where atoms are grouped. Most data can be accessed through AtomGroups.
- **Universe**: Combines all particles in an AtomGroup with a trajectory, providing access to the entire molecular system.

Working with MDAnalysis typically starts with loading data into a `Universe`.

In [21]:
# loading a trajectory
u = mda.Universe("topology.pdb", "traj.dcd")
u

/usr/local/lib/python3.12/dist-packages/MDAnalysis/coordinates/DCD.py:171: DeprecationWarning:

DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.



<Universe with 6986 atoms>

In [22]:
# Go to the last frame of the trajectory
u.trajectory[-1]

# Write the last frame to a PDB file
with mda.Writer("last_frame.pdb", u.atoms.n_atoms) as writer:
    writer.write(u.atoms)

# Visualize the last frame using py3Dmol
print("Last Frame (Full System - Protein + Water + Ions):")
view = py3Dmol.view(width=800, height=500)
with open("last_frame.pdb") as f:
    pdb_data = f.read()
    view.addModel(pdb_data, "pdb")

# Style: Protein as cartoon, water/ions as spheres
view.setStyle({"model": 0}, {"cartoon": {"color": "spectrum"}})
view.addStyle({"resn": "HOH"}, {"sphere": {"radius": 0.4}})
view.addStyle({"resn": ["NA", "CL"]}, {"sphere": {"radius": 1, "color": "yellow"}})
view.zoomTo()
view.show()

Last Frame (Full System - Protein + Water + Ions):


/usr/local/lib/python3.12/dist-packages/MDAnalysis/coordinates/PDB.py:1282: UserWarning:

Found no information for attr: 'formalcharges' Using default value of '0'



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Visualize the last frame

We can extract the last frame from the trajectory and visualize it using `py3Dmol`.

The trajectory of the simulation is saved in the `u.trajectory` attribute. It is an iterator that yields a `Timestep` object for each frame in the trajectory. The `Timestep` object contains the coordinates of all atoms in the simulation at a given time step. The `Timestep` object also contains the simulation box dimensions, the simulation time, and the simulation step.

In [23]:
# Get the number of frames in the trajectory
len(u.trajectory)

305

Get the number of residues and atoms in the protein

In [24]:
u.residues, u.atoms

(<ResidueGroup with 2171 residues>, <AtomGroup with 6986 atoms>)

MDAnalysis has powerful selection tools that allow you to select atoms based on their properties. For example:

In [25]:
# Select the first 5 residues of the protein
print(u.select_atoms("resid 1:5").residues)
# Select the residues that are within 5 angstroms of the first residue
print(u.select_atoms("around 5 resid 1").residues)
# Selec the PHE residues
print(u.select_atoms("resname PHE").residues)

<ResidueGroup [<Residue LEU, 1>, <Residue SER, 2>, <Residue ASP, 3>, ..., <Residue HOH, 5>, <Residue CL, 1>, <Residue CL, 2>]>
<ResidueGroup [<Residue SER, 2>, <Residue ASP, 3>, <Residue ASP, 5>, ..., <Residue HOH, 2051>, <Residue HOH, 2088>, <Residue HOH, 2112>]>
<ResidueGroup [<Residue PHE, 6>, <Residue PHE, 10>, <Residue PHE, 17>, <Residue PHE, 35>]>


For the `Universe` object, water molecules are considered as residues. To select only the protein, you can use the `protein` keyword.

In [26]:
# Select the protein
protein = u.select_atoms("protein")

In [27]:
type(protein)

MDAnalysis.core.groups.AtomGroup

`AtomGroups` are the core data structure in MDAnalysis. You can get the positions of the atoms in an `AtomGroup` as a numpy array, as well as other properties such as the atom names, residue names, residue numbers, and residue IDs.

In [28]:
protein.positions

array([[35.426918, 13.696164, 25.795788],
       [35.46083 , 12.83038 , 25.276773],
       [36.288048, 14.221104, 25.850554],
       ...,
       [24.504156, 20.63038 , 33.050835],
       [25.044952, 20.237331, 32.00028 ],
       [24.509933, 21.793072, 33.443253]], dtype=float32)

In [29]:
print("Center of Mass:", protein.center_of_mass())
print("Center of Geometry:", protein.center_of_geometry())
print("Total Mass:", protein.total_mass())
print("Radius of Gyration:", protein.radius_of_gyration())

Center of Mass: [22.91352224 21.04972444 23.11238122]
Center of Geometry: [22.89805297 21.06885077 23.1764342 ]
Total Mass: 4083.776
Radius of Gyration: 9.506135939864118


## Working with trajectories

The trajectory of a Universe contains the changing coordinate information.
The standard way to assess the information of each frame in a trajectory is to iterate over it. When the timestep changes, the universe only contains information associated with that timestep.



In [30]:
for ts in u.trajectory:
    time = u.trajectory.time
    rgyr = protein.radius_of_gyration()
    print(f"Frame: {ts.frame:3d}, Time: {time:4.0f} ps, Rgyr: {rgyr:.4f} A")

Frame:   0, Time:    4 ps, Rgyr: 9.7603 A
Frame:   1, Time:    8 ps, Rgyr: 9.3303 A
Frame:   2, Time:   12 ps, Rgyr: 9.2351 A
Frame:   3, Time:   16 ps, Rgyr: 9.2045 A
Frame:   4, Time:   20 ps, Rgyr: 9.0444 A
Frame:   5, Time:   24 ps, Rgyr: 9.0699 A
Frame:   6, Time:   28 ps, Rgyr: 9.1627 A
Frame:   7, Time:   32 ps, Rgyr: 9.1979 A
Frame:   8, Time:   36 ps, Rgyr: 9.1322 A
Frame:   9, Time:   40 ps, Rgyr: 9.2248 A
Frame:  10, Time:   44 ps, Rgyr: 9.2082 A
Frame:  11, Time:   48 ps, Rgyr: 9.2255 A
Frame:  12, Time:   52 ps, Rgyr: 9.1324 A
Frame:  13, Time:   56 ps, Rgyr: 9.1401 A
Frame:  14, Time:   60 ps, Rgyr: 9.0939 A
Frame:  15, Time:   64 ps, Rgyr: 9.2365 A
Frame:  16, Time:   68 ps, Rgyr: 9.1968 A
Frame:  17, Time:   72 ps, Rgyr: 9.2214 A
Frame:  18, Time:   76 ps, Rgyr: 9.2169 A
Frame:  19, Time:   80 ps, Rgyr: 9.2566 A
Frame:  20, Time:   84 ps, Rgyr: 9.4191 A
Frame:  21, Time:   88 ps, Rgyr: 9.3602 A
Frame:  22, Time:   92 ps, Rgyr: 9.2943 A
Frame:  23, Time:   96 ps, Rgyr: 9

In order to collect the radius of gyration, we can iterate over the trajectory and store the information in a list.
This can then be converted into other data structures, such as a numpy array or a pandas DataFrame. It can be plotted (as below), or used for further analysis.

In [31]:
import pandas as pd

rgyr = []
time = []
protein = u.select_atoms("protein")
for ts in u.trajectory:
    time.append(u.trajectory.time)
    rgyr.append(protein.radius_of_gyration())

rgyr_df = pd.DataFrame(rgyr, columns=["Radius of gyration (A)"], index=time)
rgyr_df.index.name = "Time (ps)"
rgyr_df

,Radius of gyration (A)
Time (ps),
4.000000,9.760273
8.000000,9.330345
12.000000,9.235060
16.000000,9.204502
20.000000,9.044394
...,...
1204.000004,9.510955
1208.000004,9.538264
1212.000004,9.453223


In [32]:
# Using pandas with plotly backend
fig = rgyr_df.plot(title="Radius of gyration")
fig.update_xaxes(title_text="Time (ps)")
fig.update_yaxes(title_text="Radius of gyration (Å)")
fig.show()

RMSD is a common metric for assessing the similarity of two structures.

In [33]:
from MDAnalysis.analysis import rms

bb = u.select_atoms("backbone")

u.trajectory[0]  # first frame
first = bb.positions

u.trajectory[-1]  # last frame
last = bb.positions

rms.rmsd(first, last)

np.float64(8.058971183377018)

In the trajectory analysis, we can calculate the RMSD of each frame to the first frame.

In [34]:
u.trajectory[0]  # set to first frame
rmsd_analysis = rms.RMSD(u, select="backbone", groupselections=["name CA", "protein"])
rmsd_analysis.run()

In [35]:
rmsd_analysis.results.rmsd.shape

(305, 5)

We can interpret this as an array with 30 rows and 5 columns. Each row is the RMSD associated with a frame in the trajectory. The columns are as follows:

1. Frame number
2. Time (ps)
3. RMSD (backbone)
4. RMSD (C-alpha)
5. RMSD (protein)

In [36]:
import pandas as pd

rmsd_df = pd.DataFrame(
    rmsd_analysis.results.rmsd[:, 2:],
    columns=["Backbone", "C-alphas", "Protein"],
    index=rmsd_analysis.results.rmsd[:, 1],
)
rmsd_df.index.name = "Time (ps)"
rmsd_df.head()

,Backbone,C-alphas,Protein
Time (ps),,,
4.0,3.948587e-07,0.000000,5.590538e-08
8.0,8.627873e-01,0.801630,1.205837e+00
12.0,1.070892e+00,1.015105,1.491666e+00
16.0,1.241954e+00,1.185400,1.677009e+00
20.0,1.553081e+00,1.503209,2.091880e+00


In [37]:
# Using pandas with plotly backend
fig = rmsd_df.plot(title="RMSD")
fig.update_xaxes(title_text="Time (ps)")
fig.update_yaxes(title_text="RMSD (Å)")
fig.show()

# For the project

This could take plenty of time, so please, start early!.

1. Use OpenMM to run a simulation of your protein of interest.
2. Execute the simulation until observe a stable RMSD and Total Energy. In real life experiments, you will need to run the simulation for a long time.  
3. Use MDAnalysis to analyze the trajectory. Calculate the RMSD of the backbone and the Total Energy of the simulation as a function of time.
4. Compare the last snapshot of the trajectory with the initial structure and the one predicted by Alphafold. Are they similar? Why or why not?

The first step could be complicated. Most of the PDBs contain errors, like missing atoms or residues.
You need to fix your PDB before you can use it with OpenMM.
You can use [PDBfixer](https://github.com/openmm/pdbfixer).

# Exercises

1. Extend the simulation, running 3 more iterations (See `Resume multiple times` subsubsection).
2. Use MDAnalysis to calculate the radius of gyration and RMSD of the protein for each frame in the trajectory. Plot the radius of gyration as a function of time.
3. Plot the RMSD of the backbone and the Total Potential Energy of the simulation as a function of time (use subplots). Are they correlated? Why or why not?
4. Use MDAnalysis to calculate the distances between sidechains of the three phenylalanines that comprise the hydrophobic core. Plot the distances as a function of time. Is the hydrophobic core stable? Why or why not?

Hint: Use labels and legends in the plots to make them more readable.


# Exercise 3

In [42]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Zeitspalte für md_log_df erzeugen
timestep_ps = 0.004  # Zeitschritt aus Simulation
md_log_df["Time (ps)"] = md_log_df["Step"] * timestep_ps

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    subplot_titles=("Backbone RMSD", "Potential Energy"),
    vertical_spacing=0.1
)

# 1) RMSD (Backbone)
fig.add_trace(
    go.Scatter(
        x=rmsd_df.index,
        y=rmsd_df["Backbone"],
        mode="lines",
        name="Backbone RMSD"
    ),
    row=1, col=1
)

# 2) Potential Energy
fig.add_trace(
    go.Scatter(
        x=md_log_df["Time (ps)"],
        y=md_log_df["Potential Energy (kJ/mole)"],
        mode="lines",
        name="Potential Energy"
    ),
    row=2, col=1
)

fig.update_xaxes(title_text="Time (ps)", row=2, col=1)
fig.update_yaxes(title_text="RMSD (Å)", row=1, col=1)
fig.update_yaxes(title_text="Potential Energy (kJ/mol)", row=2, col=1)

fig.update_layout(height=700, showlegend=True)
fig.show()

**Interpretation of the Result**

The backbone RMSD and the potential energy are not strongly correlated.

The RMSD increases while the energy remains constant, indicating that the protein undergoes structural relaxation without major energetic destabilization.

# Exercise 4

In [45]:
!grep "PHE" topology.pdb

ATOM     72  N   PHE A   6      25.339  17.866  13.494  1.00  0.00           N  
ATOM     73  H   PHE A   6      26.307  17.627  13.656  1.00  0.00           H  
ATOM     74  CA  PHE A   6      24.969  19.230  13.734  1.00  0.00           C  
ATOM     75  HA  PHE A   6      23.994  19.279  14.218  1.00  0.00           H  
ATOM     76  CB  PHE A   6      25.981  19.975  14.628  1.00  0.00           C  
ATOM     77  HB3 PHE A   6      25.740  21.022  14.447  1.00  0.00           H  
ATOM     78  HB2 PHE A   6      26.967  19.787  14.205  1.00  0.00           H  
ATOM     79  CG  PHE A   6      25.971  19.593  16.070  1.00  0.00           C  
ATOM     80  CD1 PHE A   6      26.376  20.611  16.965  1.00  0.00           C  
ATOM     81  HD1 PHE A   6      26.812  21.534  16.612  1.00  0.00           H  
ATOM     82  CE1 PHE A   6      26.409  20.458  18.330  1.00  0.00           C  
ATOM     83  HE1 PHE A   6      26.871  21.171  18.997  1.00  0.00           H  
ATOM     84  CZ  PHE A   6  

In [47]:
import MDAnalysis as mda
import numpy as np
import pandas as pd
from MDAnalysis.lib.distances import distance_array

# Trajektorie laden
u = mda.Universe("topology.pdb", "traj.dcd")

# Drei Phenylalanine, die den hydrophoben Core bilden
phe10 = u.select_atoms("resid 10 and name CG CD1 CD2 CE1 CE2 CZ")
phe17 = u.select_atoms("resid 17 and name CG CD1 CD2 CE1 CE2 CZ")
phe35 = u.select_atoms("resid 35 and name CG CD1 CD2 CE1 CE2 CZ")

# Paare definieren
pairs = [
    ("PHE10–PHE17", phe10, phe17),
    ("PHE10–PHE35", phe10, phe35),
    ("PHE17–PHE35", phe17, phe35),
]

# Arrays zum Speichern
time = []
dist_data = {label: [] for (label, _, _) in pairs}

# Distanzberechnung
for ts in u.trajectory:
    time.append(u.trajectory.time)

    for label, ag1, ag2 in pairs:
        dmat = distance_array(ag1.positions, ag2.positions)   # atom–atom matrix
        dist = np.min(dmat)                                   # minimaler Abstand
        dist_data[label].append(dist)

# In DataFrame umwandeln
dist_df = pd.DataFrame(dist_data, index=time)
dist_df.index.name = "Time (ps)"

# Plotten
fig = dist_df.plot(
    title="Distances between PHE sidechains in the hydrophobic core"
)
fig.update_xaxes(title_text="Time (ps)")
fig.update_yaxes(title_text="Distance (Å)")
fig.show()

**Interpretation of the Result**

Yes, the hydrophobic core is stable.

All three distances between the phenylalanine sidechains fluctuate only within a limited range and do not show any long-term increase or drift. The distance between PHE10 and PHE17 remains especially stable at around ~3.5. The distances involving PHE35 (PHE10–PHE35 and PHE17–PHE35) show moderate thermal fluctuations, but their average values remain constant throughout the trajectory.

These observations indicate that the phenylalanines stay closely packed and the hydrophobic core remains intact during the simulation.

For example, to create the villin.pdb file used in this tutorial, you need to execute the following code:

In [44]:
!pdbfixer YOURPROTEIN.pdb

/bin/bash: line 1: pdbfixer: command not found


You can also execute pdbfixer and it will pop up a web interface.